In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import sys
import xlsxwriter


In [2]:
top_15 = list(['TKAT','CETX'])
print(len(top_15))
for i in range(len(top_15)):
    print (top_15[i])

2
TKAT
CETX


In [3]:
strats = np.array([[200,-100], [1.05, .95], [1.1,.9],[1.15, .85], [1.2, .8], [1.2, 1.15],[200,.95],[200,.9],[200,.88],[200,.85]])
static_sells = np.array([[1.08,.85],[1.08,.88],[1.08,.91],[1.09,.91],[1.10,.91]])

print(range(len(strats)))


range(0, 10)


In [4]:
total_returns = []
sold = [] 
stopped = [] 
held = []
workbook  = xlsxwriter.Workbook('/Users/jpjansen/Documents/Stoploss3_21.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write(0,0,'Scheme')
worksheet.write(1,0,'%profit')
worksheet.write(2,0,'% of group sold')
worksheet.write(3,0,'% of group stopped')
worksheet.write(4,0,'% of group held')
worksheet.write(0,1,'hold til end of week')
worksheet.write(0,2,'5% take profit 5% trlstp')
worksheet.write(0,3,'10% profit 10% trlstp')
worksheet.write(0,4,'15% prft 15% trlstp')
worksheet.write(0,5,'20% prft 20% trlstp')
worksheet.write(0,6,'20% prft 15% trlstp')
worksheet.write(0,7,'5% trlstp')
worksheet.write(0,8,'10% trlstp')
worksheet.write(0,9,'12% trlstp')
worksheet.write(0,10,'15% trlstp')
worksheet.write(0,11,'+8%/-15% static')
worksheet.write(0,12,'+8%/-12% static')
worksheet.write(0,13,'+8%/-9% static')
worksheet.write(0,14,'+9%/-9% static')
worksheet.write(0,15,'+10%/-9% static')
worksheet.write(0,16,str(top_15))
z = 0 #strat counter
for i in range(len(strats)):
    percent_return = []
    time_sold = []
    stop_loss_stonks = []
    take_profit_stonks = []
    non_stop_loss_stonks = []
    take_profit_pcts = []
    sell_point = strats[i,0]
    stop_loss_point = strats[i,1]
    print(strats[i,:])
    for q in range(len(top_15)):
        data = yf.download(tickers=top_15[q], period="5d", interval="1m")
        iterate = data.to_numpy()
        stonk_name = top_15[q]
        runningmax = 0
        previousmaxvalue = 0
        i = 0
        for k in range(len(iterate)):
            if iterate[k,1] > runningmax:
                runningmax = iterate[k,1]
            if iterate [k,1]> sell_point*iterate[0,0]:
                print(stonk_name, "Took Profit at: ", iterate[k,1])
                take_profit_stonks.append(stonk_name)
                percent = (iterate[k,1] - iterate[0,0])/(iterate[0,0]*(len(top_15)))
                take_profit_pcts.append(percent)
                break
            if iterate[k,2] < stop_loss_point*runningmax:
                print(stonk_name, "Trailing Stop loss sell at:", iterate[k,2])
                percent = (iterate[k,2] - iterate[0,0])/(iterate[0,0]*len(top_15))
                percent_return.append(percent)
                stop_loss_stonks.append(stonk_name)
                break
            
    for i in range(len(top_15)):
        if top_15[i] not in stop_loss_stonks:
            if top_15[i] not in take_profit_stonks:
                non_stop_loss_stonks.append(top_15[i])

    stop_loss_not_met_pcts = []
    for i in range(len(non_stop_loss_stonks)):
        a = yf.download(tickers=non_stop_loss_stonks[i], period="5d", interval="1m")
        ab = a.to_numpy()
        percent = (ab[-1,3]-ab[0,0])/(ab[0,0]*len(top_15))
        stop_loss_not_met_pcts.append(percent)
    print('Held Stocks:', non_stop_loss_stonks)
    print('Took profit stocks:',take_profit_stonks)
    print('Stop loss stocks:', stop_loss_stonks)
    print((sum(stop_loss_not_met_pcts))+(sum(take_profit_pcts))+(sum(percent_return)))
    total_returns.append((sum(stop_loss_not_met_pcts))+(sum(take_profit_pcts))+(sum(percent_return)))
    complete_returns = (sum(stop_loss_not_met_pcts))+(sum(take_profit_pcts))+(sum(percent_return))
    sold.append(len(take_profit_stonks)/len(top_15))
    held.append(len(non_stop_loss_stonks)/len(top_15))
    stopped.append(len(stop_loss_stonks)/len(top_15))
    worksheet.write(1,z+1,complete_returns)
    worksheet.write(2,z+1,len(take_profit_stonks)/len(top_15))
    worksheet.write(3,z+1,len(stop_loss_stonks)/len(top_15))
    worksheet.write(4,z+1,len(non_stop_loss_stonks)/len(top_15))
    z+=1
    
static_sold = [] 
static_held = [] 
static_stopped = []
q = 0 
for x in range(len(static_sells)):
    loop_sold = []
    loop_stopped = []
    sold_pct = [] 
    stopped_pct = [] 
    held_pct = []
    held_stocks = [] 
    print(static_sells[x,:])
    for w in range(len(top_15)):
        data = yf.download(tickers=top_15[w], period="5d", interval="1m")
        iterate = data.to_numpy()
        for i in range(len(iterate)):
            if iterate[i,1]>=static_sells[x,0]*iterate[0,0]:
                print('took profit at',iterate[i,1])
                loop_sold.append(top_15[w])
                sold_pct.append((iterate[i,1] - iterate[0,0])/(iterate[0,0]*(len(top_15))))
                break
            if iterate[i,2]<=static_sells[x,1]*iterate[0,0]:
                print('sold at',iterate[i,2])
                loop_stopped.append(top_15[w])
                stopped_pct.append((iterate[i,2] - iterate[0,0])/(iterate[0,0]*len(top_15)))
                break
    for i in range(len(top_15)):
        if top_15[i] not in loop_sold:
            if top_15[i] not in loop_stopped:
                held_stocks.append(top_15[i])
    for i in range(len(held_stocks)):
        a = yf.download(tickers=held_stocks[i], period="5d", interval="1m")
        ab = a.to_numpy()
        percent = (ab[-1,3]-ab[0,0])/(ab[0,0]*len(top_15))
        held_pct.append(percent)
    static_sold.append(len(take_profit_stonks)/len(top_15))
    static_held.append(len(non_stop_loss_stonks)/len(top_15))
    static_stopped.append(len(stop_loss_stonks)/len(top_15))
    complete_returns = ((sum(stopped_pct))+(sum(sold_pct))+(sum(held_pct)))
    worksheet.write(1,q+11,complete_returns)
    worksheet.write(2,q+11,len(loop_sold)/len(top_15))
    worksheet.write(3,q+11,len(loop_stopped)/len(top_15))
    worksheet.write(4,q+11,len(held_stocks)/len(top_15))
    q+=1
workbook.close()

            

[ 200. -100.]
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Held Stocks: ['TKAT', 'CETX']
Took profit stocks: []
Stop loss stocks: []
0.16039697374406792
[1.05 0.95]
[*********************100%***********************]  1 of 1 completed
TKAT Took Profit at:  27.789899826049805
[*********************100%***********************]  1 of 1 completed
CETX Trailing Stop loss sell at: 2.359999895095825
Held Stocks: []
Took profit stocks: ['TKAT']
Stop loss stocks: ['CETX']
0.005987284143988487
[1.1 0.9]
[*********************100%***********************]  1 of 1 completed
TKAT Took Profit at:  28.799999237060547
[*********************100%***********************]  1 of 1 completed
CETX Trailing Stop loss sell at: 2.25
Held Stocks: []
Took profit stocks: ['TKAT']
Stop loss

 print(total_returns)

In [31]:
print((sum(stop_loss_not_met_pcts))+(sum(take_profit_pcts))+(sum(percent_return)))

-0.016693697522009356


In [40]:
print(take_profit_pcts)
print(sum(take_profit_pcts))

[0.004988660813652932, 0.003396226052953874]
0.008384886866606807


In [41]:
print(percent_return)

[-0.003954798664088109, -0.003934077830550547, -0.00676649899017568, -0.002754581637521197, -0.0037825023924290433, -0.00039215647775547706, -0.006438832030958564, -0.003654780448022998, -0.002222226637380919, -0.004115227306482054, -0.003696098388381753, -0.0009523800873701952]


In [42]:
print(stop_loss_not_met_pcts)

[-0.0018278390062093878]


In [18]:
blah = yf.download(tickers='SINT', period="5d", interval="1m")
np.set_printoptions(threshold=sys.maxsize)
print(blah)

[*********************100%***********************]  1 of 1 completed
                             Open    High     Low   Close  Adj Close  Volume
Datetime                                                                    
2021-02-12 09:30:00-05:00  2.9100  2.9100  2.8800  2.8800     2.8800   66616
2021-02-12 09:31:00-05:00  2.8832  2.9027  2.8400  2.8421     2.8421   18226
2021-02-12 09:32:00-05:00  2.8400  2.8800  2.8400  2.8800     2.8800   19712
2021-02-12 09:33:00-05:00  2.8800  2.8800  2.8304  2.8790     2.8790   17415
2021-02-12 09:34:00-05:00  2.8600  2.8601  2.8490  2.8490     2.8490   12422
...                           ...     ...     ...     ...        ...     ...
2021-02-19 15:55:00-05:00  2.7750  2.7750  2.7700  2.7700     2.7700     717
2021-02-19 15:56:00-05:00  2.7750  2.7750  2.7700  2.7700     2.7700    4822
2021-02-19 15:57:00-05:00  2.7720  2.7750  2.7700  2.7700     2.7700     542
2021-02-19 15:58:00-05:00  2.7800  2.7950  2.7800  2.7900     2.7900   11655
2021-02

In [19]:
print(blah)
blah = np.asarray(blah)
print(blah)
min_before_max = np.amin(blah[0:np.argmax(blah[:,1]),2])
print(min_before_max)

                             Open    High     Low   Close  Adj Close  Volume
Datetime                                                                    
2021-02-12 09:30:00-05:00  2.9100  2.9100  2.8800  2.8800     2.8800   66616
2021-02-12 09:31:00-05:00  2.8832  2.9027  2.8400  2.8421     2.8421   18226
2021-02-12 09:32:00-05:00  2.8400  2.8800  2.8400  2.8800     2.8800   19712
2021-02-12 09:33:00-05:00  2.8800  2.8800  2.8304  2.8790     2.8790   17415
2021-02-12 09:34:00-05:00  2.8600  2.8601  2.8490  2.8490     2.8490   12422
...                           ...     ...     ...     ...        ...     ...
2021-02-19 15:55:00-05:00  2.7750  2.7750  2.7700  2.7700     2.7700     717
2021-02-19 15:56:00-05:00  2.7750  2.7750  2.7700  2.7700     2.7700    4822
2021-02-19 15:57:00-05:00  2.7720  2.7750  2.7700  2.7700     2.7700     542
2021-02-19 15:58:00-05:00  2.7800  2.7950  2.7800  2.7900     2.7900   11655
2021-02-19 15:59:00-05:00  2.7850  2.8100  2.7800  2.7991     2.7991   32146

In [ ]:
print